In [95]:
import pandas as pd
import numpy as np
import geopandas as gpd
import sqlite3
import os
from fractions import Fraction

def get_ratio(ins,outs):    
    """ Function to display in to out ratio in the format 1:7 """
    if ins==0:
        ratio='0'+':'+str(int(outs))
        return ratio
    elif outs==0:
        ratio=str(int(ins))+':'+'0'
        return ratio
    elif ins==outs:
        ratio='1:1'
        return ratio
    else:
        ratio=str(Fraction(ins/outs).limit_denominator(1000)).split('/')[0]+':'+str(Fraction(ins/outs).limit_denominator(1000)).split('/')[1]
        return ratio

In [96]:
data_path='/Users/anastasiaclark/irs_migration/irs_nyc_migration/data'
db='irsmig_county_database'
con = sqlite3.connect(os.path.join(data_path,db,"irs_migration_county.sqlite"))

In [97]:
years=['2011_12','2012_13','2013_14','2014_15'] # project years 

#for year in years:
year=years[3]
table1='outflow_{}'.format(year)
table2='inflow_{}'.format(year)

In [98]:
df_out = pd.read_sql_query("SELECT * from {} where {}.origin!={}.destination".format(table1, table1, table1), con)
df_in=pd.read_sql_query("SELECT * from {} where {}.origin!={}.destination".format(table2, table2, table2), con)      

In [99]:
# Make the index to be same for same records
df_out['uid']=df_out.origin+"_"+df_out.destination
df_in['uid']=df_in.origin+"_"+df_in.destination
df_in.set_index('uid', inplace=True)
df_out.set_index('uid', inplace=True)

In [100]:
metros=pd.read_csv(os.path.join(data_path,'metros','metros_basic.csv'),converters={'fips':str,'co_code':str,'cbsa_code':str})
metros.head()
metros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 8 columns):
cbsa_code      1825 non-null object
cbsa_name      1825 non-null object
metro_micro    1825 non-null object
co_name        1825 non-null object
st_name        1825 non-null object
st_code        1825 non-null int64
co_code        1825 non-null object
fips           1825 non-null object
dtypes: int64(1), object(7)
memory usage: 114.1+ KB


In [101]:
# NYC counties
nyc=['36005','36047','36061','36081','36085']

In [102]:
nyc_in=df_in[df_in['destination'].isin(nyc)]
nyc_out=df_out[df_out['origin'].isin(nyc)]

In [103]:
to_nyc=nyc_in[['origin','co_orig_name','exemptions','st_orig_abbrv']].groupby(['origin','co_orig_name','st_orig_abbrv']).sum().sort_values('exemptions',ascending=False).reset_index()
from_nyc=nyc_out[['destination','co_dest_name','st_dest_abbrv','exemptions']].groupby(['destination','co_dest_name','st_dest_abbrv']).sum().sort_values('exemptions',ascending=False).reset_index()
to_nyc.rename(columns={'origin':'co_fips','co_orig_name':'co_name','exemptions':'inflow','st_orig_abbrv':'state'},inplace=True)
from_nyc.rename(columns={'destination':'co_fips','co_dest_name':'co_name','st_dest_abbrv':'state','exemptions':'outflow'},inplace=True)

In [104]:
flows_nyc = to_nyc.merge(from_nyc, on=['co_fips', 'co_name', 'state'], how='outer')
flows_nyc.sort_values('inflow', ascending=False).head(n=10)

co_fips                        co_name state   inflow  outflow
0   36047                   Kings County    NY  41886.0  36031.0
1   36061                New York County    NY  38830.0  28352.0
2   36081                  Queens County    NY  27061.0  33000.0
3   59000  Other flows - Different State    DS  19743.0  23807.0
4   36005                   Bronx County    NY  19504.0  26424.0
5   36059                  Nassau County    NY  11900.0  20316.0
6   36119             Westchester County    NY   9048.0  13986.0
7   36103                 Suffolk County    NY   5837.0   8171.0
8   34017                  Hudson County    NJ   4709.0   7273.0
9   36085                Richmond County    NY   4234.0   7708.0

In [105]:
flows_nyc.sort_values('outflow',ascending=False).head(n=10)
# top senders to NYC are also top receivers of migrabts from NYC; first 4 counties are ranked the same for recieving and sending

co_fips                        co_name state   inflow  outflow
0   36047                   Kings County    NY  41886.0  36031.0
2   36081                  Queens County    NY  27061.0  33000.0
1   36061                New York County    NY  38830.0  28352.0
4   36005                   Bronx County    NY  19504.0  26424.0
3   59000  Other flows - Different State    DS  19743.0  23807.0
5   36059                  Nassau County    NY  11900.0  20316.0
6   36119             Westchester County    NY   9048.0  13986.0
7   36103                 Suffolk County    NY   5837.0   8171.0
9   36085                Richmond County    NY   4234.0   7708.0
8   34017                  Hudson County    NJ   4709.0   7273.0

In [106]:
# calculate fractions, net flow and ratio
flows_nyc['net_flow'] = flows_nyc.inflow - flows_nyc.outflow
flows_nyc['in_ratio'] = flows_nyc.inflow / flows_nyc.outflow
flows_nyc['out_ratio'] = flows_nyc.outflow / flows_nyc.inflow
# need to fill Nulls with 0 for in and out flow to apply get_ration function
flows_nyc.inflow.fillna(0, inplace=True)
flows_nyc.outflow.fillna(0, inplace=True)
flows_nyc['in_to_out_ratio'] = flows_nyc.apply(lambda x: get_ratio(x['inflow'], x['outflow']), axis=1)
flows_nyc.head()

co_fips                        co_name state   inflow  outflow  net_flow  \
0   36047                   Kings County    NY  41886.0  36031.0    5855.0   
1   36061                New York County    NY  38830.0  28352.0   10478.0   
2   36081                  Queens County    NY  27061.0  33000.0   -5939.0   
3   59000  Other flows - Different State    DS  19743.0  23807.0   -4064.0   
4   36005                   Bronx County    NY  19504.0  26424.0   -6920.0   

   in_ratio  out_ratio in_to_out_ratio  
0  1.162499   0.860216           93:80  
1  1.369568   0.730157           63:46  
2  0.820030   1.219467         524:639  
3  0.829294   1.205845         787:949  
4  0.738117   1.354799         730:989

In [107]:
# drop a column in each table, so that the columns are the same in both tables
df_in.drop('co_orig_name',1,inplace=True)
df_out.drop('co_dest_name',1,inplace=True)

In [108]:
# get inflow and outflow into a single table and since most of the records exist in both tables, drop duplicates
flows=pd.concat([df_in,df_out],axis=0).drop_duplicates(subset=['origin','destination','returns','exemptions'])

In [109]:
# check if there are any records with duplicated indexes; should be none
flows[flows.index.duplicated()]

destination  disclosure  exemptions    income origin  returns  \
uid                                                                         
06001_06083       06083         NaN       112.0    2469.0  06001     74.0   
06013_06085       06085         NaN      1359.0   70854.0  06013    788.0   
06013_32031       32031         NaN       261.0   22906.0  06013    141.0   
06013_06019       06019         NaN       186.0    3576.0  06013     84.0   
06013_48113       48113         NaN       101.0    5569.0  06013     51.0   
06017_06071       06071         NaN        35.0    2632.0  06017     22.0   
06029_06075       06075         NaN        59.0    4878.0  06029     40.0   
06033_06013       06013         NaN        51.0    2110.0  06033     27.0   
06037_12061       12061         NaN        41.0   10657.0  06037     23.0   
06071_53061       53061         NaN       106.0    2530.0  06071     59.0   
06071_08031       08031         NaN        54.0    1268.0  06071     35.0   
06075_04013       04013         NaN       182.0   18119.0  06075    145.0   
06075_06065       06065         NaN       184.0   14042.0  06075    142.0   
06075_12011       12011         NaN        55.0    2776.0  06075     42.0   
06075_27053       27053         NaN        47.0    3082.0  06075     38.0   
06075_06029       06029         NaN        40.0    1452.0  06075     34.0   
06075_12099       12099         NaN        45.0   10197.0  06075     32.0   
06075_15009       15009         NaN        47.0    2029.0  06075     32.0   
06075_36103       36103         NaN        31.0    2029.0  06075     27.0   
06075_53011       53011         NaN        37.0    2346.0  06075     26.0   
06075_41029       41029         NaN        29.0    1305.0  06075     20.0   
06075_48085       48085         NaN        43.0    2837.0  06075     20.0   
06075_12095       12095         NaN        23.0    1329.0  06075     19.0   
06079_06053       06053         NaN       212.0    4990.0  06079    119.0   
06079_06107       06107         NaN       118.0    2356.0  06079     62.0   
06079_06041       06041         NaN        32.0    1660.0  06079     21.0   
06081_06079       06079         NaN        73.0   86704.0  06081     50.0   
06083_06041       06041         NaN        34.0    1462.0  06083     24.0   
06085_06079       06079         NaN       255.0     659.0  06085    158.0   
06097_06075       06075         NaN       502.0  136158.0  06097    410.0   
...                 ...         ...         ...       ...    ...      ...   
34039_42101       42101         NaN       107.0   26263.0  34039     71.0   
36029_12057       12057         NaN       112.0    6466.0  36029     60.0   
36047_06013       06013         NaN        96.0   11686.0  36047     62.0   
36059_12057       12057         NaN       122.0    1647.0  36059     77.0   
36059_12061       12061         NaN        41.0   10441.0  36059     24.0   
36081_06013       06013         NaN        78.0    9304.0  36081     42.0   
36085_12057       12057         NaN        59.0    3512.0  36085     29.0   
36119_12061       12061         NaN        35.0   10749.0  36119     20.0   
37063_42101       42101         NaN        50.0    3785.0  37063     35.0   
37063_06075       06075         NaN        30.0    4933.0  37063     29.0   
37119_06075       06075         NaN        59.0   11878.0  37119     40.0   
40123_40125       40125         NaN        70.0    2349.0  40123     34.0   
41005_06075       06075         NaN        32.0    2643.0  41005     28.0   
41051_42101       42101         NaN        35.0    3003.0  41051     25.0   
42017_06075       06075         NaN        35.0    9798.0  42017     30.0   
42075_42101       42101         NaN        59.0    -744.0  42075     42.0   
42089_42101       42101         NaN        78.0    8194.0  42089     62.0   
42091_24003       24003         NaN        60.0    6902.0  42091     34.0   
42101_36081       36081         NaN       312.0    7512.0  42101    192.0   
42129_12

In [110]:
# examine some of the records that might be errors

# why there are records that have different exemptions/income numbers in inflow and outflow tables
# where they should be the same?
# Run a check in the sqlite databse--same results
# SELECT * FROM inflow_2014_15 where origin="06001" and destination ="06083"
# SELECT * FROM outflow_2014_15 where origin="06001" and destination ="06083"
flows.loc['06013_06085']

destination  disclosure  exemptions    income origin  returns  \
uid                                                                         
06013_06085       06085         NaN      1363.0  256538.0  06013    789.0   
06013_06085       06085         NaN      1359.0   70854.0  06013    788.0   

            st_dest_abbrv st_orig_abbrv  
uid                                      
06013_06085            CA            CA  
06013_06085            CA            CA

# Continue from here after above issue resolved

In [111]:
# metro_in=df_in[(df_in.cbsa_name.notnull) & (df_in.cbsa_name.str.contains('New York'))].copy()
# metro_in.rename(columns={'cbsa_name':'cbsa_dest'},inplace=True)

# metro_out=df_out[(df_out.cbsa_name.notnull) & (df_out.cbsa_name.str.contains('New York'))].copy()
# metro_out.rename(columns={'cbsa_name':'cbsa_orig'},inplace=True)

In [112]:
# merge metro area information
# df_in = df_in.merge(metros[['fips', 'cbsa_name']],left_on='destination', right_on='fips', how='left').drop('fips',1).rename(columns={'cbsa_name':'dest_cbsa'})
# df_in = df_in.merge(metros[['fips', 'cbsa_name']],left_on='origin', right_on='fips', how='left').rename(columns={'cbsa_name':'orig_cbsa'})

# df_out = df_out.merge(metros[['fips', 'cbsa_name']],left_on='destination', right_on='fips', how='left').drop('fips',1).rename(columns={'cbsa_name':'dest_cbsa'})
# df_out = df_out.merge(metros[['fips', 'cbsa_name']],left_on='origin', right_on='fips', how='left').rename(columns={'cbsa_name':'orig_cbsa'})

In [113]:
# group in and out flows by the metro area
# sort in the descending order of inflow first, then outflow
# by_metro = flows_nyc[['cbsa_name', 'inflow', 'outflow']].groupby('cbsa_name').sum().sort_values(['inflow', 'outflow'], ascending=False)
# top 10 metro areas that send/recieve to/from NYC counties
# by_metro.head(n=10)

In [114]:
con.close()